# Voldemort's Stable Diffusion WebUI

Adapted from: https://colab.research.google.com/drive/1Iy-xW9t1-OQWhb0hNxueGij8phCyluOh

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded dependencies will be auto-updated.

In [ ]:
!nvidia-smi

### Clone the central repository

In [ ]:
import os.path

if not os.path.exists('/notebooks/stable-diffusion-webui'):
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else: # update repo if already exists
    print('stable-diffusion-webui already downloaded, updating')
    %cd /notebooks/stable-diffusion-webui
    !git pull

### Install requirements and download repositories.

In [ ]:
%cd /notebooks/stable-diffusion-webui

!pip install -r requirements.txt

if not os.path.exists('/notebooks/stable-diffusion-webui/repositories'): # download repositories if they don't exist
    !mkdir /notebooks/stable-diffusion-webui/repositories
    !git clone https://github.com/CompVis/stable-diffusion.git /notebooks/stable-diffusion-webui/repositories/stable-diffusion
    !git clone https://github.com/CompVis/taming-transformers.git /notebooks/stable-diffusion-webui/repositories/taming-transformers
    !git clone https://github.com/sczhou/CodeFormer.git /notebooks/stable-diffusion-webui/repositories/CodeFormer
else: # update repositories if they do exist
    print('Updating dependencies')
    for dir in os.listdir('/notebooks/stable-diffusion-webui/repositories'):
        %cd /notebooks/stable-diffusion-webui/repositories/$dir
        !git pull
    
!pip install -r /notebooks/stable-diffusion-webui/repositories/CodeFormer/requirements.txt

if not os.path.exists('/notebooks/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
else:
    print('GFPGANv1.3.pth already downloaded')

In [ ]:
# import os
# os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

### Download the model

**Pick only one!**

#### Standard Model

Voldy provided an alternative download if you don't want to use HuggingFace.

[https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view](https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view)

Download it to your computer, create the directory `/notebooks/stable-diffusion/` in this notebook (pro-tip: use `mkdir -p`), then upload it to that dir (make sure it's named `model.ckpt`).

HuggingFace is much faster and reliable but you need to get access to the repo and provide your user token.

In [ ]:
user_token = "<enter user_token here>"

# ===============================================================================================

if not os.path.exists('/notebooks/stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt'):
    user_header = f"'Authorization: Bearer {user_token}'"
    !wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /notebooks/stable-diffusion-webui/model.ckpt
else:
    print('Standard model already downloaded')

#### Waifu Diffusion

**HTTPS**

In [ ]:
if not os.path.exists('/notebooks/stable-diffusion-webui/model.ckpt'):
    !wget https://thisanimedoesnotexist.ai/downloads/wd-v1-2-full-ema.ckpt -O /notebooks/stable-diffusion-webui/model.ckpt
else:
    print('Waifu Diffusion model already downloaded')

**Gradient Dataset**

In [ ]:
mounted_dataset_path = '/datasets/<name>'
!rsync -ah --info=progress2 $mounted_dataset_path/wd-v1-2-full-ema.ckpt /notebooks/stable-diffusion-webui/model.ckpt

# Launch the WebUI

You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

In [ ]:
import sys
# sys.argv = ["--opt-split-attention"] # low memory optimizations, not nessesary for Paperspace.
sys.argv = [''] # if you aren't using any args you must provide an empty array.

import webui
import modules.ui
import modules.txt2img
import modules.img2img

demo = modules.ui.create_ui(
    txt2img=webui.wrap_gradio_gpu_call(modules.txt2img.txt2img),
    img2img=webui.wrap_gradio_gpu_call(modules.img2img.img2img),
    run_extras=webui.wrap_gradio_gpu_call(webui.run_extras),
    run_pnginfo=webui.run_pnginfo
)

demo.launch(share=True)